# Assignment 3 260587899 Lim, Dongjoon

## Import libraries and define useful functions

In [1]:
from scipy.sparse import hstack
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
from collections import Counter
import collections, re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline

def F1score(classifier, testFeature, testTarget, trainFeature, trainTarget, validFeature, validTarget, name):
    classifier.fit(trainFeature,trainTarget)
    predicted_test = classifier.predict(testFeature)
    predicted_train =classifier.predict(trainFeature)
    predicted_valid =classifier.predict(validFeature)
    f1_test = f1_score(testTarget,predicted_test, average='macro')
    f1_train = f1_score(trainTarget,predicted_train, average='macro')
    f1_valid = f1_score(validTarget,predicted_valid, average='macro')
    #bestParam = classifier.best_params_
    print('Below is f1 scores and best parameter for '+str(name))
    print('f1 score for yelp {} classifier test is: {}'.format(name, f1_test))
    print('f1 score for yelp {} classifier train is: {}'.format(name, f1_train))
    print('f1 score for yelp {} classifier valid is: {}'.format(name, f1_valid))
    print('====================================================================================')
    #print('Best parameter is: '.format(bestParam))
    return f1_valid
def F1score2(classifier, testFeature, testTarget, trainFeature, trainTarget, validFeature, validTarget, name):
    classifier.fit(trainFeature,trainTarget)
    bestParam = classifier.best_params_
    result= classifier.cv_results_
    predicted_test = classifier.predict(testFeature)
    predicted_train =classifier.predict(trainFeature)
    predicted_valid =classifier.predict(validFeature)
    f1_test = f1_score(testTarget,predicted_test, average='macro')
    f1_train = f1_score(trainTarget,predicted_train, average='macro')
    f1_valid = f1_score(validTarget,predicted_valid, average='macro')
    
    print('Below is f1 scores and best parameter for '+str(name))
    print('f1 score for yelp {} classifier test is: {}'.format(name, f1_test))
    print('f1 score for yelp {} classifier train is: {}'.format(name, f1_train))
    print('f1 score for yelp {} classifier valid is: {}'.format(name, f1_valid))
    print('Best parameter is: {}'.format(bestParam))
    print('====================================================================================')
    return f1_valid

def F1score3(classifier, testFeature, testTarget, trainFeature, trainTarget, validFeature, validTarget, name):
    classifier.fit(trainFeature,trainTarget)
    bestParam = classifier.best_params_
    predicted_test = classifier.predict(testFeature)
    predicted_train =classifier.predict(trainFeature)
    predicted_valid =classifier.predict(validFeature)
    f1_test = f1_score(testTarget,predicted_test, average='macro')
    f1_train = f1_score(trainTarget,predicted_train, average='macro')
    f1_valid = f1_score(validTarget,predicted_valid, average='macro')
    
    print('Below is f1 scores and best parameter for '+str(name))
    print('f1 score for imdb {} classifier test is: {}'.format(name, f1_test))
    print('f1 score for imdb {} classifier train is: {}'.format(name, f1_train))
    print('f1 score for imdb {} classifier valid is: {}'.format(name, f1_valid))
    print('Best parameter for {} is: {}'.format(name, bestParam))
    print('====================================================================================')
    return f1_valid
def F1score4(classifier, testFeature, testTarget, trainFeature, trainTarget, validFeature, validTarget, name):
    classifier.fit(trainFeature,trainTarget)
    predicted_test = classifier.predict(testFeature)
    predicted_train =classifier.predict(trainFeature)
    predicted_valid =classifier.predict(validFeature)
    f1_test = f1_score(testTarget,predicted_test, average='macro')
    f1_train = f1_score(trainTarget,predicted_train, average='macro')
    f1_valid = f1_score(validTarget,predicted_valid, average='macro')
    #bestParam = classifier.best_params_
    print('Below is f1 scores and best parameter for '+str(name))
    print('f1 score for imdb {} classifier test is: {}'.format(name, f1_test))
    print('f1 score for imdb {} classifier train is: {}'.format(name, f1_train))
    print('f1 score for imdb {} classifier valid is: {}'.format(name, f1_valid))
    print('====================================================================================')
    #print('Best parameter is: '.format(bestParam))
    return f1_valid

def saveVocab(wordList, frequencyList, name):
    with open('Generated_Data/{}-vocab.txt'.format(name),'w+') as file:
        for i in range(10000):
            file.write('{}\t{}\t{}\n'.format(wordList[i],i,frequencyList[i]))

def savedata(vocabulary, data ,name, datatype):
    with open('Generated_Data/{}-{}.txt'.format(name, datatype),'w+') as file:
        row =0
        for sentence in data[0]:
            line = ''
            token = sentence.split(' ')
            for piece in token:
                id = vocabulary.get(piece)
                #ignore if the word is not frequent.
                if id != None:
                    line += str(id)+ ' '
            file.write('{}\t{}\n'.format(line, data[1][row])) 
            row = row+1

yelp_test = pd.read_csv('Data/yelp-test.txt', header=None, sep="\t")
yelp_train = pd.read_csv('Data/yelp-train.txt', header=None, sep="\t")
yelp_valid = pd.read_csv('Data/yelp-valid.txt', header=None, sep="\t")

imdb_train = pd.read_csv('Data/IMDB-train.txt', header=None, sep="\t")
imdb_valid = pd.read_csv('Data/IMDB-valid.txt', header=None, sep="\t")
imdb_test = pd.read_csv('Data/IMDB-test.txt', header=None, sep="\t")

#Sanity check
yelp_train.head()
imdb_train.head()

,0,1
0,For a movie that gets no respect there sure ar...,1
1,Bizarre horror movie filled with famous faces ...,1
2,"A solid, if unremarkable film. Matthau, as Ein...",1
3,It's a strange feeling to sit alone in a theat...,1
4,"You probably all already know this by now, but...",1


## Q1 Preprocess data and vectorize

In [2]:
#Removing punctuations and change capital to lower case
yelp_train[0] = yelp_train[0].str.replace('<br /><br />', ' ').str.replace('[^\w\s]', '').str.lower()
yelp_test[0] = yelp_test[0].str.replace('<br /><br />', ' ').str.replace('[^\w\s]', '').str.lower()
yelp_valid[0] = yelp_valid[0].str.replace('<br /><br />', ' ').str.replace('[^\w\s]', '').str.lower()
imdb_train[0] = imdb_train[0].str.replace('<br /><br />', ' ').str.replace('[^\w\s]', '').str.lower()
imdb_test[0] = imdb_test[0].str.replace('<br /><br />', ' ').str.replace('[^\w\s]', '').str.lower()
imdb_valid[0] = imdb_valid[0].str.replace('<br /><br />', ' ').str.replace('[^\w\s]', '').str.lower()

#Make bag of words
vectorizer = CountVectorizer(max_features = 10000)

yelpTrain = vectorizer.fit_transform(yelp_train[0])
yelpTest = vectorizer.transform(yelp_test[0])
yelpValid = vectorizer.transform(yelp_valid[0])

vocabulary = vectorizer.vocabulary_
wordList = vectorizer.get_feature_names()
frequencyList = np.asarray(yelpTrain.sum(axis=0))[0]

saveVocab(wordList, frequencyList, 'yelp')
savedata(vocabulary, yelp_train ,'yelp', 'train')
savedata(vocabulary, yelp_test ,'yelp', 'test')
savedata(vocabulary, yelp_valid ,'yelp', 'valid')

imdbTrain = vectorizer.fit_transform(imdb_train[0])
imdbTest = vectorizer.transform(imdb_test[0])
imdbValid = vectorizer.transform(imdb_valid[0])

vocabulary = vectorizer.vocabulary_
wordList = vectorizer.get_feature_names()
frequencyList = np.asarray(yelpTrain.sum(axis=0))[0]

saveVocab(wordList, frequencyList, 'IMDB')
savedata(vocabulary, imdb_train ,'IMDB', 'train')
savedata(vocabulary, imdb_test ,'IMDB', 'test')
savedata(vocabulary, imdb_valid ,'IMDB', 'valid')

#Normalize to get frequency bag of words that frequencies sums up to 1
normal = Normalizer()

yelpTrain_freq = normal.transform(yelpTrain)
yelpTrain_freq = yelpTrain_freq/yelpTrain_freq.sum()
yelpTest_freq = normal.transform(yelpTest)
yelpTest_freq = yelpTest_freq/yelpTest_freq.sum()
yelpValid_freq = normal.transform(yelpValid)
yelpValid_freq = yelpValid_freq/yelpValid_freq.sum()
imdbTrain_freq = normal.transform(imdbTrain)
imdbTrain_freq = imdbTrain_freq/imdbTrain_freq.sum()
imdbTest_freq = normal.transform(imdbTest)
imdbTest_freq = imdbTest_freq/imdbTest_freq.sum()
imdbValid_freq = normal.transform(imdbValid)
imdbValid_freq = imdbValid_freq/imdbValid_freq.sum()
print(type(yelpTrain))



#sanity check
#print(yelpTrain_freq)

<class 'scipy.sparse.csr.csr_matrix'>


## Q2, train models and perform hyperparameter tuning to get f1 measures. Binary Bag of words

In [3]:
uniform = DummyClassifier(strategy='uniform', random_state=100) #set random seed so we get consistent results
frequent = DummyClassifier(strategy='most_frequent')
Nbayes_grid = [
    {
        'alpha' : np.arange(0.01, 0.3, 0.01)
    }
]
Nbayes = GridSearchCV(BernoulliNB(), Nbayes_grid, cv=5,scoring='f1_micro', refit = True)

pipeline = make_pipeline(
     FunctionTransformer(lambda x: x.todense(), accept_sparse=True), 
     GaussianNB()
)
gauss_grid = {}
GaussNbayes = GridSearchCV(pipeline, gauss_grid, cv=5,scoring='f1_micro', refit = True)

dt_grid = [
    {
        'criterion':['gini','entropy'], 
        'max_leaf_nodes': [18,19,20,21,22],
        'max_depth':[8,9,10,11,12],
        'min_samples_split':[2,3,4,5,6]
    }
]
DecisionTree = GridSearchCV(DecisionTreeClassifier(), dt_grid, cv=5,scoring='f1_micro', refit = True)

svc_grid = [
    {
        'max_iter': [15 * i for i in range(5)],
        'loss':['hinge','squared_hinge'],
        'C':[0.5,1.0,2.0,3.0]
    }
]
svc = GridSearchCV(LinearSVC(), svc_grid, cv=5,scoring='f1_micro', refit = True)
#print(type(yelpTest))
#print(type(yelp_test[1]))
F1score(uniform, yelpTest, yelp_test[1], yelpTrain, yelp_train[1], yelpValid, yelp_valid[1], 'uniform')
F1score(frequent, yelpTest, yelp_test[1], yelpTrain, yelp_train[1], yelpValid, yelp_valid[1], 'majority frequent')

F1score2(Nbayes, yelpTest, yelp_test[1], yelpTrain, yelp_train[1], yelpValid, yelp_valid[1], 'Naive Bayes')
F1score2(DecisionTree, yelpTest, yelp_test[1], yelpTrain, yelp_train[1], yelpValid, yelp_valid[1], 'Decision Tree')
F1score2(svc, yelpTest, yelp_test[1], yelpTrain, yelp_train[1], yelpValid, yelp_valid[1], 'Linear Support Vector Machine')

Below is f1 scores and best parameter for uniform
f1 score for yelp uniform classifier test is: 0.17520594816848462
f1 score for yelp uniform classifier train is: 0.18336811298204822
f1 score for yelp uniform classifier valid is: 0.1880367210174599
Below is f1 scores and best parameter for majority frequent
f1 score for yelp majority frequent classifier test is: 0.10392301998519615
f1 score for yelp majority frequent classifier train is: 0.10426700464723279
f1 score for yelp majority frequent classifier valid is: 0.10501474926253689


/home/dongjoon/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Below is f1 scores and best parameter for Naive Bayes
f1 score for yelp Naive Bayes classifier test is: 0.38115077661846924
f1 score for yelp Naive Bayes classifier train is: 0.7292359932480867
f1 score for yelp Naive Bayes classifier valid is: 0.3702751079974438
Best parameter is: {'alpha': 0.09}


/home/dongjoon/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Below is f1 scores and best parameter for Decision Tree
f1 score for yelp Decision Tree classifier test is: 0.23318797392174656
f1 score for yelp Decision Tree classifier train is: 0.2742832668464026
f1 score for yelp Decision Tree classifier valid is: 0.26490158061164243
Best parameter is: {'criterion': 'gini', 'max_depth': 8, 'max_leaf_nodes': 19, 'min_samples_split': 2}
Below is f1 scores and best parameter for Linear Support Vector Machine
f1 score for yelp Linear Support Vector Machine classifier test is: 0.4075051599519378
f1 score for yelp Linear Support Vector Machine classifier train is: 0.9875560114861116
f1 score for yelp Linear Support Vector Machine classifier valid is: 0.4358022431846056
Best parameter is: {'C': 0.5, 'loss': 'squared_hinge', 'max_iter': 60}


0.4358022431846056

## Q3, train models and perform hyperparameter tuning to get f1 measures. Frequent Bag of words

In [4]:
F1score2(GaussNbayes, yelpTest_freq, yelp_test[1], yelpTrain_freq, yelp_train[1], yelpValid_freq, yelp_valid[1], 'Naive Bayes')
F1score2(DecisionTree, yelpTest_freq, yelp_test[1], yelpTrain_freq, yelp_train[1], yelpValid_freq, yelp_valid[1], 'Decision Tree')
F1score2(svc, yelpTest_freq, yelp_test[1], yelpTrain_freq, yelp_train[1], yelpValid_freq, yelp_valid[1], 'Linear Support Vector Machine')

Below is f1 scores and best parameter for Naive Bayes
f1 score for yelp Naive Bayes classifier test is: 0.24399044080292537
f1 score for yelp Naive Bayes classifier train is: 0.722856822871117
f1 score for yelp Naive Bayes classifier valid is: 0.2841059259835029
Best parameter is: {}


/home/dongjoon/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Below is f1 scores and best parameter for Decision Tree
f1 score for yelp Decision Tree classifier test is: 0.24299026355849002
f1 score for yelp Decision Tree classifier train is: 0.26996428201753997
f1 score for yelp Decision Tree classifier valid is: 0.2502904291095359
Best parameter is: {'criterion': 'gini', 'max_depth': 8, 'max_leaf_nodes': 22, 'min_samples_split': 2}
Below is f1 scores and best parameter for Linear Support Vector Machine
f1 score for yelp Linear Support Vector Machine classifier test is: 0.25408289598519107
f1 score for yelp Linear Support Vector Machine classifier train is: 0.23189004677273753
f1 score for yelp Linear Support Vector Machine classifier valid is: 0.23966125643654448
Best parameter is: {'C': 1.0, 'loss': 'hinge', 'max_iter': 45}


0.23966125643654448

## Q4, do the same thing to imdb dataset Bag of words

In [5]:
F1score4(uniform, imdbTest, imdb_test[1], imdbTrain, imdb_train[1], imdbValid, imdb_valid[1], 'uniform')
F1score4(frequent, imdbTest, imdb_test[1], imdbTrain, imdb_train[1], imdbValid, imdb_valid[1], 'majority frequent')

F1score3(Nbayes, imdbTest, imdb_test[1], imdbTrain, imdb_train[1], imdbValid, imdb_valid[1], 'Naive Bayes')
F1score3(DecisionTree, imdbTest, imdb_test[1], imdbTrain, imdb_train[1], imdbValid, imdb_valid[1], 'Decision Tree')
F1score3(svc, imdbTest, imdb_test[1], imdbTrain, imdb_train[1], imdbValid, imdb_valid[1], 'Linear Support Vector Machine')

Below is f1 scores and best parameter for uniform
f1 score for imdb uniform classifier test is: 0.5014310369266095
f1 score for imdb uniform classifier train is: 0.49586407652832204
f1 score for imdb uniform classifier valid is: 0.4954868776136868
Below is f1 scores and best parameter for majority frequent
f1 score for imdb majority frequent classifier test is: 0.3333333333333333
f1 score for imdb majority frequent classifier train is: 0.3333333333333333
f1 score for imdb majority frequent classifier valid is: 0.3333333333333333


/home/dongjoon/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Below is f1 scores and best parameter for Naive Bayes
f1 score for imdb Naive Bayes classifier test is: 0.8359549446896211
f1 score for imdb Naive Bayes classifier train is: 0.874624832595253
f1 score for imdb Naive Bayes classifier valid is: 0.8452683589267169
Best parameter for Naive Bayes is: {'alpha': 0.2}
Below is f1 scores and best parameter for Decision Tree
f1 score for imdb Decision Tree classifier test is: 0.7243986733167429
f1 score for imdb Decision Tree classifier train is: 0.7307857226619265
f1 score for imdb Decision Tree classifier valid is: 0.7202767199353735
Best parameter for Decision Tree is: {'criterion': 'gini', 'max_depth': 12, 'max_leaf_nodes': 22, 'min_samples_split': 2}
Below is f1 scores and best parameter for Linear Support Vector Machine
f1 score for imdb Linear Support Vector Machine classifier test is: 0.8513956269719808
f1 score for imdb Linear Support Vector Machine classifier train is: 0.96198960981761
f1 score for imdb Linear Support Vector Machine cl

0.8636309631751073

## Q5, do the same thing to imdb dataset frequency Bag of words

In [6]:
F1score3(GaussNbayes, imdbTest_freq, imdb_test[1], imdbTrain_freq, imdb_train[1], imdbValid_freq, imdb_valid[1], 'Naive Bayes')
F1score3(DecisionTree, imdbTest_freq, imdb_test[1], imdbTrain_freq, imdb_train[1], imdbValid_freq, imdb_valid[1], 'Decision Tree')
F1score3(svc, imdbTest_freq, imdb_test[1], imdbTrain_freq, imdb_train[1], imdbValid_freq, imdb_valid[1], 'Linear Support Vector Machine')

Below is f1 scores and best parameter for Naive Bayes
f1 score for imdb Naive Bayes classifier test is: 0.6388643134391188
f1 score for imdb Naive Bayes classifier train is: 0.8685870455970421
f1 score for imdb Naive Bayes classifier valid is: 0.7765259764644441
Best parameter for Naive Bayes is: {}
Below is f1 scores and best parameter for Decision Tree
f1 score for imdb Decision Tree classifier test is: 0.41541847944062177
f1 score for imdb Decision Tree classifier train is: 0.41959800798879177
f1 score for imdb Decision Tree classifier valid is: 0.42078762338391085
Best parameter for Decision Tree is: {'criterion': 'gini', 'max_depth': 12, 'max_leaf_nodes': 18, 'min_samples_split': 3}
Below is f1 scores and best parameter for Linear Support Vector Machine
f1 score for imdb Linear Support Vector Machine classifier test is: 0.6624099731565676
f1 score for imdb Linear Support Vector Machine classifier train is: 0.6718681799783213
f1 score for imdb Linear Support Vector Machine classifi

0.6624601146460078